In [1]:
import pandas as pd
import requests
from config import api_key
import json

In [2]:
dem_candidate=pd.read_csv("dem_candidates_2020.csv")
dem_finance=pd.read_csv("2020_dem_finance.csv")

In [3]:
dem_candidate=dem_candidate.rename(columns={
    "state":"candidate_state"
})
dem_finance=dem_finance.rename(columns={
    "state":"donor_state"
})

In [4]:
dem_data=pd.DataFrame.merge(dem_candidate, dem_finance, on="candidate_id")
dem_data.head(1)

,Unnamed: 0_x,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,party.1,candidate_status,Unnamed: 0_y,donor_state,donation_count,total_raised,cycle
0,1,"AHLERS, DAN",S0SD00070,SD,DEM,Challenger,Senate,[2020],[2020],DEM,C,0,CA,2,2000.0,2020


In [5]:
dem_data=dem_data.filter(["name","candidate_id","candidate_state","party","incumbent_challenge_full","office_full","cycles","election_years",
                        "candidate_status","donor_state","donation_count","total_raised"])
dem_data.shape

(2378, 12)

In [6]:
in_state=(dem_data.candidate_state==dem_data.donor_state)
in_state=in_state.to_frame()
in_state=in_state.rename(columns={
    0: "in_state"
})
in_state.head()

,in_state
0,False
1,False
2,True
3,False
4,False


In [7]:
dem_data=pd.DataFrame.merge(dem_data,in_state, left_index=True, right_index=True)

dem_data.set_index("candidate_id")

,name,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,candidate_status,donor_state,donation_count,total_raised,in_state
candidate_id,,,,,,,,,,,,
S0SD00070,"AHLERS, DAN",SD,DEM,Challenger,Senate,[2020],[2020],C,CA,2,2000.0,False
S0SD00070,"AHLERS, DAN",SD,DEM,Challenger,Senate,[2020],[2020],C,DC,2,3800.0,False
S0SD00070,"AHLERS, DAN",SD,DEM,Challenger,Senate,[2020],[2020],C,SD,118,26585.0,True
S0SD00070,"AHLERS, DAN",SD,DEM,Challenger,Senate,[2020],[2020],C,WY,2,1000.0,False
S0CO00500,"BAER, DAN",CO,DEM,Challenger,Senate,[2020],[2020],C,AZ,3,1500.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
S0SC00305,"TINUBU, GLORIA BROMELL DR.",SC,DEM,Challenger,Senate,[2020],[2020],C,MA,1,500.0,False
S0SC00305,"TINUBU, GLORIA BROMELL DR.",SC,DEM,Challenger,Senate,[2020],[2020],C,MN,1,500.0,False
S0SC00305,"TINUBU, GLORIA BROMELL DR.",SC,DEM,Challenger,Senate,[2020],[2020],C,SC,7,4760.0,True


In [8]:
aggregate=dem_data.groupby(["candidate_id", "in_state"]).agg({"donation_count":["sum"], "total_raised":["sum"]})



aggregate.columns = aggregate.columns.droplevel(1)

aggregate=aggregate.reset_index()

in_state=aggregate.loc[aggregate["in_state"]==True]
in_state
out_of_state=aggregate.loc[aggregate["in_state"]==False]
out_of_state

state_agg=pd.DataFrame.merge(in_state, out_of_state, on="candidate_id", suffixes=("_in_state","_out_of_state"), how="outer")
# state_agg.to_csv("dem_finance_data.csv")
state_agg

,candidate_id,in_state_in_state,donation_count_in_state,total_raised_in_state,in_state_out_of_state,donation_count_out_of_state,total_raised_out_of_state
0,S0AL00156,True,6869.0,1569818.78,False,28772.0,7150753.95
1,S0AR00200,True,287.0,87820.00,False,81.0,26305.00
2,S0AZ00350,True,49292.0,6322262.65,False,99114.0,18429967.21
3,S0CO00286,True,13853.0,1851289.40,False,1440.0,314913.15
4,S0CO00435,True,354.0,45220.88,False,75.0,20260.70
...,...,...,...,...,...,...,...
88,S8NE00216,True,58.0,18153.19,False,15.0,6995.00
89,S8OR00207,True,27811.0,1834286.77,False,10805.0,1151278.54
90,S8TN00311,True,4258.0,1187668.04,False,754.0,462207.96
91,S0KY00388,NaN,NaN,NaN,False,1.0,500.00


In [9]:
final_dem_data=pd.DataFrame.merge(dem_candidate,state_agg, on="candidate_id")

In [11]:
final_dem_data_2020=final_dem_data.filter(["name", "candidate_id", "candidate_state", "party", "incumbent_challenge_full", "cycles", "donation_count_in_state", "total_raised_in_state","donation_count_out_of_state", "total_raised_out_of_state"])

In [23]:
final_dem_data_2020=final_dem_data_2020.filter(["name", "candidate_id", "candidate_state", "party", "incumbent_challenge_full", "cycles", "donation_count_in_state", "total_raised_in_state","donation_count_out_of_state", "total_raised_out_of_state"])
final_dem_data_2020

,name,candidate_id,candidate_state,party,incumbent_challenge_full,cycles,donation_count_in_state,total_raised_in_state,donation_count_out_of_state,total_raised_out_of_state
0,"AHLERS, DAN",S0SD00070,SD,DEM,Challenger,[2020],118.0,26585.00,6.0,6800.00
1,"BAER, DAN",S0CO00500,CO,DEM,Challenger,[2020],1138.0,285837.59,1225.0,837717.00
2,"BALLARD, JACK",S0MT00116,MT,DEM,Challenger,[2020],13.0,4630.00,29.0,16450.00
3,"BELL, ROBERT CHRISTOPHER MR.",S0TX00324,TX,DEM,Challenger,[2020],537.0,312254.40,41.0,30609.00
4,"BEN DAVID, MERAV",S0WY00152,WY,DEM,Open seat,[2020],26.0,10785.00,33.0,15367.39
...,...,...,...,...,...,...,...,...,...,...
88,"SWEARENGIN, PAULA JEAN",S0WV00215,WV,DEM,Challenger,[2020],201.0,12206.00,1106.0,210064.47
89,"SWEET, BETSY",S0ME00103,ME,DEM,Challenger,[2020],1761.0,197807.34,1378.0,204638.47
90,"TARVER, EDWARD J.",S0GA00583,GA,DEM,Challenger,[2020],173.0,93284.81,52.0,24596.72
91,"TERRY, EDWARD",S0GA00419,GA,DEM,Challenger,[2020],168.0,50937.00,43.0,11705.00


In [30]:
final_dem_data_2020.to_csv("final_dem_data_2020.csv")

In [31]:
gop=pd.read_csv("2020_final_rep_data.csv")
gop=gop.rename(columns={"state": "candidate_state"})

In [32]:
to_concat=[final_dem_data_2020,gop]

In [33]:
all_candidates=pd.concat(to_concat)
all_candidates

,name,candidate_id,candidate_state,party,incumbent_challenge_full,cycles,donation_count_in_state,total_raised_in_state,donation_count_out_of_state,total_raised_out_of_state,Unnamed: 0,Unnamed: 0.1,office_full,election_years,party.1,candidate_status,in_state_in_state,in_state_out_of_state
0,"AHLERS, DAN",S0SD00070,SD,DEM,Challenger,[2020],118.0,26585.00,6.0,6800.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"BAER, DAN",S0CO00500,CO,DEM,Challenger,[2020],1138.0,285837.59,1225.0,837717.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"BALLARD, JACK",S0MT00116,MT,DEM,Challenger,[2020],13.0,4630.00,29.0,16450.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"BELL, ROBERT CHRISTOPHER MR.",S0TX00324,TX,DEM,Challenger,[2020],537.0,312254.40,41.0,30609.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"BEN DAVID, MERAV",S0WY00152,WY,DEM,Open seat,[2020],26.0,10785.00,33.0,15367.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,"TUCKER, GARLAND S III",S0NC00319,NC,REP,Challenger,[2020],727.0,440605.76,49.0,46225.00,97.0,15.0,Senate,[2020],REP,C,True,False
98,"WAGLE, SUSAN",S0KS00299,KS,REP,Open seat,[2020],242.0,306690.47,84.0,45825.00,98.0,16.0,Senate,[2020],REP,C,True,False
99,"WATERS, ALLEN",S0RI00067,RI,REP,Challenger,[2020],10.0,4510.00,2.0,3050.00,99.0,17.0,Senate,[2020],REP,C,True,False
100,"WHEELER, JOSHUA B",S0WY00103,WY,REP,Open seat,[2020],7.0,5375.00,3.0,700.00,100.0,18.0,Senate,[2020],REP,C,True,False


In [34]:
all_candidates=all_candidates.filter(["name","candidate_id","candidate_state","party","incumbent_challenge_full", "cycles", "donation_count_in_state", "total_raised_in_state","donation_count_out_of_state","total_raised_out_of_state" ])

In [35]:
all_candidates.to_csv("2020_final_candidates.csv")